In [21]:
import gensim.corpora as corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import unicodedata
import pyLDAvis

# Lista de stop words personalizadas y comunes en español
stopwords = [
    'ir', 'hacer', 'año', 'ser', 'decir', 'siento', 'millon', 'mil', 'ahora',
    'mexicanos', 'y', 'el', 'hoy', 'aqui', 'mexico', 'dar', 'solo', 
    'todo', 'deber', 'cada', 'de', 'la', 'en', 'un', 'que', 'a', 'los', 
    'las', 'con', 'se', 'para', 'por', 'no', 'su', 'este', 'es', 'lo', 
    'como', 'mas', 'pero', 'al', 'del', 'mi', 'te', 'me', 'si', 'ya', 
    'cuando', 'otra', 'vez', 'hay', 'estaba', 'tambien', 'asi', 'mismo', 'él',
    'mexicano', 'dia', 'ano', 'dos', 'senor'
]

# Función para normalizar el texto
def normalize_text(text):
    # Eliminar acentos y convertir el texto a minúsculas
    text = ''.join(
        char for char in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(char)
    ).lower()
    
    # Dividir el texto en palabras
    words = text.split()
    
    # Eliminar stop words personalizadas (incluyendo 'él' y otros)
    words = [word for word in words if word not in stopwords]
    
    return words

# Cargar el texto preprocesado
with open("Enrique_Peñanieto_Procesado.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Dividir el texto en documentos (por ejemplo, por párrafos o líneas)
raw_documents = text.split('\n')  # Asume que cada línea es un documento
documents = [normalize_text(doc) for doc in raw_documents if doc.strip()]  # Preprocesar cada documento

# Crear el diccionario y el corpus
id2word = corpora.Dictionary(documents)  # Crear diccionario
corpus = [id2word.doc2bow(doc) for doc in documents]  # Crear el corpus

# Entrenar el modelo LDA
num_topics = 3  # Ajusta el número de tópicos según tus necesidades
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42)

# Evaluar la coherencia del modelo
coherence_model_lda = CoherenceModel(model=lda_model, texts=documents, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f"Coherencia del modelo: {coherence_lda}")

# Visualizar los tópicos con pyLDAvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'lda_visualization_Peña Nieto.html')
print("La visualización LDA se ha guardado en 'lda_visualization_Peña Nieto.html'.")

# Paso 9: Crear y guardar nubes de palabras para cada tópico
for idx in range(num_topics):
    words = dict(lda_model.show_topic(idx, topn=20))
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(words)
    
    # Crear la figura
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Tópico {idx + 1}")
    
    # Guardar la imagen
    filename = f"nube_palabras_topico_Peña Nieto_{idx + 1}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()  # Cerrar la figura para liberar memoria

    print(f"Nube de palabras del Tópico {idx + 1} guardada como {filename}.")

Coherencia del modelo: 0.15146011401832946
La visualización LDA se ha guardado en 'lda_visualization_Peña Nieto.html'.
Nube de palabras del Tópico 1 guardada como nube_palabras_topico_Peña Nieto_1.png.
Nube de palabras del Tópico 2 guardada como nube_palabras_topico_Peña Nieto_2.png.
Nube de palabras del Tópico 3 guardada como nube_palabras_topico_Peña Nieto_3.png.
